<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [68]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.31 s


In [69]:
import math
import numpy as np
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == 'EUC_2D':
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

time: 18 ms


In [70]:
lines = open("burma14.tsp", "r").readlines()
index_start_coordinates = 0
cont = 0
V = []

for line in lines:
  cont += 1
  if line.replace(" ", "").startswith("EOF"):
    break
  elif line.replace(" ", "").startswith("DIMENSION"):
    n = int(line.split(":")[1][1:])
  elif line.replace(" ", "").startswith("EDGE_WEIGHT_TYPE"):
    t = line.replace(" ", "").split(":")[1][1:]
  elif line.replace(" ", "").startswith("NODE_COORD_SECTION"):
    index_start_coordinates = cont
  elif index_start_coordinates > 0:
    V.append((int(line.split()[0]) - 1, [float(line.split()[1]), float(line.split()[2])])) #(i, [x_value, y_value])

#n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
#t = lines[4].split()[1]

time: 13 ms


In [71]:
# build a unique identifier for the subset S, joining the indexes of the vertexes with a blank space between them
def encode(S):
  encoded_string = ""
  for s in S:
    encoded_string += " " + str(s[0])
  return encoded_string

subsets = {} # dictionary to enumerate the subsets
counter = 0
subsets[encode(V)] = counter # add the first subset, with all vertexes
counter += 1

d = np.zeros(shape = (n, 2 ** (n - 1))) # d[v, S]: distance of the TSP starting from 0 to v, passing through al points of S
phi = np.zeros(shape = (n, 2 ** (n - 1))) # phi[v, S]: predecessor of v 

time: 14 ms


In [72]:
import time
def held_karp (v, S, max_time): # v: arrival vertex of S starting from 0, S: subset of vertexes 
  S_index = subsets[encode(S)] # build a unique identifier for the subset S
  if time.time() > max_time: # max time: 3 min
    return None
  elif (len(S) == 1) & (S[0][0] == v): # base case: the solution is the weight of the edge {v, 0}
    return weight(V[v][1], V[0][1], t)
  elif d[v, S_index] != 0: # distance already computed
    return d[v, S_index] 
  else:  # recursive case: find the minimum among all the sub-paths
    mindist = math.inf
    minprec = None
    subset = [i for i in S if i[0] != v] # S \ {v} 
    if encode(subset) not in subsets: 
      global counter
      subsets[encode(subset)] = counter
      counter += 1
    for u in subset:
      dist = held_karp(u[0], subset, max_time) # compute the partial result
      if dist is None:
        break
      else:
        w = weight(u[1], V[v][1], t)
        if (dist + w) < mindist:
          mindist = dist + w
          minprec = u[0]
    d[v, S_index] = mindist # update d with the minimum distance
    phi[v, S_index] = minprec # update phi with predecessor of v
    return mindist

time: 17.7 ms


In [73]:
start = time.time()
max_time = 120
print(held_karp(0, V, (start + max_time)))

3323.0
time: 2.15 s
